In [2]:
import numpy as np

E=210e6 
mu=0.3
t=0.025 
w=3000 

nodes=np.array([[0,0],[0.5,0],[0.5,0.25],[0,0.25],[0.25,0.125]])
Connections=np.array([[1,5,4],[5,3,4],[5,2,3],[1,2,5]])-1     
no_of_elements=Connections.shape[0]
no_of_dof=nodes.shape[0]*2
dofs=np.array([(1,2),(3,4),(5,6),(7,8),(9,10)])-1
dof_seq=np.array([1,2,7,8,3,4,5,6,9,10])-1
free_dof=6 
Elenodes=np.ndarray((no_of_elements, 3,2))  
for i in range(Connections.shape[0]):
    for j in range(Connections.shape[1]):
        Elenodes[i,j,0]=nodes[Connections[i][j]][0]
        Elenodes[i,j,1]=nodes[Connections[i][j]][1]
        
###Cacculates the individual element stiffness matrix of a traingular element

def LinearTriangleElementStiffness(E,mu,t,p,EleNumber=1):
    
    xi,yi,xj,yj,xm,ym=Elenodes[i].flatten()
    A = (xi*(yj-ym)+ xj*(ym-yi) + xm*(yi-yj))/2
    bi = yj-ym
    bj = ym-yi
    bm = yi-yj
    gi = xm-xj
    gj = xi-xm
    gm = xj-xi
    B=np.asarray([[bi, 0 ,bj, 0, bm, 0],
                  [0, gi, 0, gj, 0 ,gm],
                  [gi,bi,gj,bj,gm,bm]])

    B=B*(1/(2*A))
    if p==1:                                              #plane stress condition
        D=(E/(1-mu*mu))*np.asarray([[1,mu,0],[mu,1,0],[0,0,(1-mu)/2]])
    elif p==2:                                             #plane strain condition
        D=(E/((1+mu)*(1-2*mu)))*np.asarray([[1-mu,mu,0],[mu,1-mu,0],[0,0,(1-2*mu)/2]])
    
    return t*A*np.dot(B.T,np.dot(D,B))




In [6]:
#calcaute individual element stiffness matrix and store it in single array km
km=np.ndarray((no_of_elements,6,6))
for i in range(no_of_elements):
    km[i]=LinearTriangleElementStiffness(E,mu,t,1,EleNumber=i)

print('\n------Element Stifness Matrix-------:=\n',km)

# GOLABL MATRIX
K = np.zeros((8, 8))
def LinearTriangleAssemble(K :np.ndarray, k, nodes):
    i,j,m=nodes
    
    K[2*i-1-1, 2*i-1-1] = K[2*i-1-1, 2*i-1-1] + k[1-1, 1-1]
    K[2*i-1-1, 2*i-1] = K[2*i-1-1,  2*i-1] + k[1-1, 2-1]
    K[2*i-1-1, 2*j-1-1] = K[2*i-1-1,  2*j-1-1] + k[1-1, 3-1]
    K[2*i-1-1,  2*j-1] = K[2*i-1-1,  2*j-1] + k[1-1, 4-1]
    K[2*i-1-1,  2*m-1-1] = K[2*i-1-1,  2*m-1-1] + k[1-1, 5-1]
    K[2*i-1-1,  2*m-1] = K[2*i-1-1,  2*m-1] + k[1-1, 6-1]
    K[2*i-1, 2*i-1-1] = K[2*i-1, 2*i-1-1] + k[2-1, 1-1]
    K[2*i-1, 2*i-1] = K[2*i-1, 2*i-1] + k[2-1, 2-1]
    K[2*i-1, 2*j-1-1] = K[2*i-1, 2*j-1-1] + k[2-1, 3-1]
    K[2*i-1, 2*j-1] = K[2*i-1, 2*j-1] + k[2-1, 4-1]
    K[2*i-1, 2*m-1-1] = K[2*i-1, 2*m-1-1] + k[2-1, 5-1]
    K[2*i-1, 2*m-1] = K[2*i-1, 2*m-1] + k[2-1, 6-1]
    K[2*j-1-1, 2*i-1-1] = K[2*j-1-1,  2*i-1-1] + k[3-1, 1-1]
    K[2*j-1-1,  2*i-1] = K[2*j-1-1,  2*i-1] + k[3-1, 2-1]
    K[2*j-1-1,  2*j-1-1] = K[2*j-1-1,  2*j-1-1] + k[3-1, 3-1]
    K[2*j-1-1,  2*j-1] = K[2*j-1-1,  2*j-1] + k[3-1, 4-1]
    K[2*j-1-1,  2*m-1-1] = K[2*j-1-1,  2*m-1-1] + k[3-1, 5-1]
    K[2*j-1-1,  2*m-1] = K[2*j-1-1, 2*m-1] + k[3-1, 6-1]
    K[2*j-1, 2*i-1-1] = K[2*j-1, 2*i-1-1] + k[4-1, 1-1]
    K[2*j-1, 2*i-1] = K[2*j-1, 2*i-1] + k[4-1, 2-1]
    K[2*j-1, 2*j-1-1] = K[2*j-1, 2*j-1-1] + k[4-1, 3-1]
    K[2*j-1, 2*j-1] = K[2*j-1, 2*j-1] + k[4-1, 4-1]
    K[2*j-1, 2*m-1-1] = K[2*j-1, 2*m-1-1] + k[4-1, 5-1]
    K[2*j-1, 2*m-1] = K[2*j-1, 2*m-1] + k[4-1, 6-1]
    K[2*m-1-1,  2*i-1-1] = K[2*m-1-1,  2*i-1-1] + k[5-1, 1-1]
    K[2*m-1-1,  2*i-1] = K[2*m-1-1, 2*i-1] + k[5-1, 2-1]
    K[2*m-1-1,  2*j-1-1] = K[2*m-1-1,  2*j-1-1] + k[5-1, 3-1]
    K[2*m-1-1,  2*j-1] = K[2*m-1-1,  2*j-1] + k[5-1, 4-1]
    K[2*m-1-1,  2*m-1-1] = K[2*m-1-1,  2*m-1-1] + k[5-1, 5-1]
    K[2*m-1-1, 2*m-1] = K[2*m-1-1,  2*m-1] + k[5-1, 6-1]
    K[2*m-1, 2*i-1-1] = K[2*m-1, 2*i-1-1] + k[6-1, 1-1]
    K[2*m-1, 2*i-1] = K[2*m-1, 2*i-1] + k[6-1, 2-1]
    K[2*m-1, 2*j-1-1] = K[2*m-1, 2*j-1-1] + k[6-1, 3-1]
    K[2*m-1, 2*j-1] = K[2*m-1, 2*j-1] + k[6-1, 4-1]
    K[2*m-1, 2*m-1-1] = K[2*m-1, 2*m-1-1] + k[6-1, 5-1]
    K[2*m-1, 2*m-1] = K[2*m-1, 2*m-1] + k[6-1, 6-1]
    
    return K

np.set_printoptions(linewidth=np.inf)
KG=np.zeros((no_of_dof,no_of_dof))
for i in range(no_of_elements):
    nodes=Connections[i]+1
    KG=LinearTriangleAssemble(KG,km[i],nodes)

print('\n------Global Stifness Matrix-------:=\n',KG)


------Element Stifness Matrix-------:=
 [[[ 1730769.23076923   937500.         -1442307.69230769 -1009615.38461538  -288461.53846154    72115.38461538]
  [  937500.          3137019.23076923  -865384.61538462  -504807.69230769   -72115.38461538 -2632211.53846154]
  [-1442307.69230769  -865384.61538462  2884615.38461538        0.         -1442307.69230769   865384.61538462]
  [-1009615.38461538  -504807.69230769        0.          1009615.38461538  1009615.38461538  -504807.69230769]
  [ -288461.53846154   -72115.38461538 -1442307.69230769  1009615.38461538  1730769.23076923  -937500.        ]
  [   72115.38461538 -2632211.53846154   865384.61538462  -504807.69230769  -937500.          3137019.23076923]]

 [[ 4038461.53846154        0.         -2019230.76923077 -1009615.38461538 -2019230.76923077  1009615.38461538]
  [       0.         11538461.53846154  -865384.61538462 -5769230.76923077   865384.61538462 -5769230.76923077]
  [-2019230.76923077  -865384.61538462  1730769.23076923   93

In [7]:
F=np.zeros((no_of_dof,))
U=np.zeros((no_of_dof,))
#Applying Boundary conditions
U=np.zeros((no_of_dof,1))
F=np.zeros((no_of_dof,1))
#LOADING CONDITIONS
F[2,0]=9.375
F[3,0]=0
F[4,0]=9.375
F[5,0]=0
F[8,0]=0
F[9,0]=0
dof_seq=np.array([3,4,5,6,9,10,1,2,7,8])-1 
xx=free_dof
#Matrix condensation
Up=U[dof_seq[:xx],]
Ux=U[dof_seq[xx:],] 
Fp=F[dof_seq[:xx],]
Kpp=KG[np.ix_(dof_seq[:xx],dof_seq[:xx])]
print('\n------Known nodal forces-------:=\n',Fp)
Up=np.dot(np.linalg.inv(Kpp),Fp)
print('\n----------Unknown nodal disps----------:=\n',Up)
#Adding Calcuated unknown displacements to the main displacement vector
U[dof_seq[:xx],]=Up
#Calculating force
F=np.dot(KG,U)
print('----------FORCES VECTOR(kN)-------------:=\n',np.round(F,7))
##Array which contains displacement(nodal) of a traingular element as rows.
u=np.ndarray((no_of_elements,6))
for p in range(no_of_elements):
    xi=dofs[Connections[p][0]][0]
    yi=dofs[Connections[p][0]][1]
    
    xj=dofs[Connections[p][1]][0]
    yj=dofs[Connections[p][1]][1]
    
    xm=dofs[Connections[p][2]][0]
    ym=dofs[Connections[p][2]][1]
    
    u[p]=U[[xi,yi,xj,yj,xm,ym],].T
    


------Known nodal forces-------:=
 [[9.375]
 [0.   ]
 [9.375]
 [0.   ]
 [0.   ]
 [0.   ]]

----------Unknown nodal disps----------:=
 [[ 6.92834261e-06]
 [ 7.13904355e-07]
 [ 6.92834261e-06]
 [-7.13904355e-07]
 [ 3.27082221e-06]
 [ 4.23516474e-22]]
----------FORCES VECTOR(kN)-------------:=
 [[-9.375    ]
 [-3.7540043]
 [ 9.375    ]
 [ 0.       ]
 [ 9.375    ]
 [-0.       ]
 [-9.375    ]
 [ 3.7540043]
 [-0.       ]
 [ 0.       ]]


In [ ]:
#Element stresses
def LinearTriangleElementStresses(E,mu,p=1,EleNumber=1):
    xi,yi,xj,yj,xm,ym=Elenodes[i].flatten()
    uu=u[EleNumber]
    
    A = (xi*(yj-ym)+ xj*(ym-yi) + xm*(yi-yj))/2
    bi = yj-ym
    bj = ym-yi
    bm = yi-yj
    gi = xm-xj
    gj = xi-xm
    gm = xj-xi
    B=np.asarray([[bi, 0 ,bj, 0, bm, 0],
                  [0, gi, 0, gj, 0 ,gm],
                  [gi,bi,gj,bj,gm,bm]])

    B=B*(1/(2*A))
    
    if p==1:                                    #plane stress condition
        D=(E/(1-mu*mu))*np.asarray([[1,mu,0],
                                    [mu,1,0],
                                    [0,0,(1-mu)/2]])
        
    elif p==2:                                 #plane strain condition
        D=(E/((1+mu)*(1-2*mu)))*np.asarray([[1-mu,mu,0],
                                            [mu,1-mu,0],
                                            [0,0,(1-2*mu)/2]])

    return np.dot(D,np.dot(B,uu)).reshape(np.dot(D,np.dot(B,uu)).shape[0],1)



In [8]:
# FUNCTION TO CALCUATE PRINCIPLE STRESSSES
def LinearTriangleElementPStresses(sigma):
    R = (sigma[0] + sigma[1])/2
    Q = ((sigma[0] - sigma[1])/2)**2 + sigma[2]*sigma[2]
    M = 2*sigma[2]/(sigma[0] - sigma[1])
    s1 = R + np.sqrt(Q)
    s2 = R - np.sqrt(Q)
    theta = (np.arctan(M)/2)*180/np.pi
    return np.asarray([s1 , s2 , theta])


sigma=np.ndarray((no_of_elements,3,1))
for i in range(no_of_elements):
    sigma[i]=LinearTriangleElementStresses(E,mu,EleNumber=i)

for i in range(no_of_elements):
    print(f'\nStress:ELEMENT{i+1}\n\n {sigma[i]}')

prinStress=np.ndarray(sigma.shape)
for i in range(no_of_elements):
    prinStress[i]= LinearTriangleElementPStresses(sigma[i])
# print(prinStress)

for i in range(no_of_elements):
    print(f'\nPrincipal Stress:=ELEMENT{i+1}\n\n {prinStress[i]}')


Stress:ELEMENT1

 [[3.01922050e+03]
 [9.05766151e+02]
 [1.36828399e-13]]

Stress:ELEMENT2

 [[3000.        ]
 [ 300.3203417 ]
 [   9.61025093]]

Stress:ELEMENT3

 [[2980.77949813]
 [-305.12546716]
 [   0.        ]]

Stress:ELEMENT4

 [[3000.        ]
 [ 300.3203417 ]
 [  -9.61025093]]

Principal Stress:=ELEMENT1

 [[3.01922050e+03]
 [9.05766151e+02]
 [3.70941998e-15]]

Principal Stress:=ELEMENT2

 [[3.00003421e+03]
 [3.00286132e+02]
 [2.03956612e-01]]

Principal Stress:=ELEMENT3

 [[2980.77949813]
 [-305.12546716]
 [   0.        ]]

Principal Stress:=ELEMENT4

 [[ 3.00003421e+03]
 [ 3.00286132e+02]
 [-2.03956612e-01]]
